In [16]:
import pandas as pd   
from matplotlib import pyplot as plt
from numpy import mean
import time

from sklearn.model_selection import train_test_split, RepeatedKFold, GridSearchCV, cross_val_score
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.preprocessing import RobustScaler

import warnings 
warnings.filterwarnings('ignore')


from sklearn.linear_model import LinearRegression

In [17]:
df1 = pd.read_csv("train.csv")
#df1.head()
df2 = pd.read_csv('test.csv')
#df2

In [18]:
#from df1 drop sub_are
df1 = df1.drop(columns=['sub_area'])


In [19]:
#from df2 drop sub_area and row ID 
df2 = df2.drop(columns=['sub_area', 'row ID'])

In [20]:
df_onehot = pd.get_dummies(df1)
df_onehot.dtypes
test_onehot = pd.get_dummies(df2)

In [21]:
target_variable = 'price_doc'  # replace 'target' with the actual name of your target variable column

correlation_with_target = df_onehot.corr()[[target_variable]].abs()


low_correlation_columns = correlation_with_target[correlation_with_target[target_variable] < 0.5].index.tolist()

print("Columns with Absolute Correlation < 0. with", target_variable, ":")
print(low_correlation_columns)

Columns with Absolute Correlation < 0. with price_doc :
['area_m', 'raion_popul', 'green_zone_part', 'indust_part', 'children_preschool', 'preschool_education_centers_raion', 'children_school', 'school_education_centers_raion', 'healthcare_centers_raion', 'sport_objects_raion', 'additional_education_raion', 'shopping_centers_raion', 'full_all', 'male_f', 'female_f', 'young_all', 'young_male', 'young_female', 'work_all', 'work_male', 'work_female', 'ekder_all', 'ekder_male', 'ekder_female', '0_6_all', '0_6_male', '0_6_female', '7_14_all', '7_14_male', '7_14_female', '0_17_all', '0_17_male', '0_17_female', '16_29_all', '16_29_male', '16_29_female', '0_13_all', '0_13_male', '0_13_female', 'raion_build_count_with_material_info', 'build_count_block', 'build_count_wood', 'build_count_frame', 'build_count_brick', 'build_count_panel', 'build_count_mix', 'raion_build_count_with_builddate_info', 'build_count_1921-1945', 'build_count_1946-1970', 'build_count_1971-1995', 'ID_metro', 'metro_min_avt

In [ ]:
df_onehot = df_onehot.drop(columns=low_correlation_columns)

In [ ]:
X = df_onehot.drop(columns=['price_doc'])
print(X.shape)
y = df_onehot[['price_doc']]




In [ ]:
#do robust scaling
scaler = RobustScaler()
scaled_df2 = scaler.fit_transform(test_onehot)
scaled_df1 = scaler.fit_transform(X)

In [ ]:
scaler = StandardScaler()

# Fit and transform the training data
scaled_df1 = scaler.fit_transform(X)

# Transform the test data using the same scaler
scaled_df2 = scaler.transform(test_onehot)

In [ ]:
scaler = MinMaxScaler()

# Fit the scaler on the training data and transform both training and test data
df_onehot_scaled = scaler.fit_transform(X)
test_onehot_scaled = scaler.transform(df2)

In [ ]:
print(scaled_df1.shape)

In [ ]:
#cpnvert to dataframe
scaled_df1 = pd.DataFrame(scaled_df1, columns=X.columns)
scaled_df2 = pd.DataFrame(scaled_df2, columns=test_onehot.columns)

In [ ]:
from sklearn.decomposition import PCA
n_components = 102  # You can choose the number of components based on your requirements
pca = PCA(n_components=n_components)

# Apply PCA to the training data
X_pca = pca.fit_transform(scaled_df1)

# Apply the same PCA transformation to the test data
df2_onehot_pca = pca.transform(scaled_df2)

In [ ]:
from sklearn.feature_selection import VarianceThreshold

# original_feature_indices = np.arange(X.shape[1])
threshold_value = 0.001
variance_filter = VarianceThreshold(threshold=threshold_value)

# Fit and transform the feature matrix
X_filtered = variance_filter.fit_transform(scaled_df1)

X_test_high_variance = variance_filter.transform(scaled_df2)


In [ ]:
import statsmodels.api as sm
import pandas as pd

# Convert scaled_df1 to a DataFrame without specifying columns
scaled_df1_df = pd.DataFrame(scaled_df1)

# Add a constant term to the feature matrix
X_with_const = sm.add_constant(scaled_df1_df)

# Fit a linear regression model
model = sm.OLS(y, X_with_const).fit()

# Get p-values for each feature
p_values = model.pvalues[1:]  # Exclude the constant term

# Set your desired threshold for p-value
threshold = 0.05

# Filter features based on p-value
selected_features = p_values[p_values < threshold].index

# Display selected features
print("Selected Features:")
print(selected_features)

# Select columns in the DataFrame
X_selected = scaled_df1_df[selected_features]


In [ ]:
print(X_selected.shape)  

In [ ]:
print(scaled_df1_selected.shape)

In [ ]:
scaled_df1_selected_df = pd.DataFrame(scaled_df1_selected)

In [ ]:
#give me code for feature importance using linear regression
model = LinearRegression()
model.fit(X, y)
# get importance
importance = model.coef_[0]
# summarize feature importance
for i,v in enumerate(importance):
    print('Feature: %0d, Score: %.5f' % (i,v))

# drop features with low importance
threshold = 0  # Adjust the threshold as needed
selected_features = [feature for feature, score in zip(X.columns, importance) if score >= threshold]
X_selected = X[selected_features]

# plot feature importance
plt.bar([x for x in range(len(importance))], importance)
plt.show()



In [ ]:
ols_reg = LinearRegression()
sfs = SequentialFeatureSelector(ols_reg, direction='forward', n_features_to_select=150)
sfs.fit(X, y)
X_selected = sfs.transform(X)
# Print the names of the selected features
print("Selected Features:", sfs.get_feature_names_out())




In [ ]:
print(X_selected.shape)

In [ ]:
X_selected_df = pd.DataFrame(X_selected, columns=sfs.get_feature_names_out())

In [ ]:
print(scaled_df1_selected_df.head())

In [ ]:
#keep features from scaled_df2 that are in X_selected
test_onehot = scaled_df2[:, scaled_df1_selected_df.columns.get_indexer(scaled_df1_selected_df.columns)]


In [ ]:
print(test_onehot.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(scaled_df1_selected, y, test_size = 0.3)

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
print("Training Set Info:")
print(X.info())

print("\nTest Set Info:")
print(test_onehot.info())


In [ ]:
reg2 = LinearRegression().fit(scaled_df1_selected_df, y)
print(reg2.score(scaled_df1_selected_df, y))
print(reg2.coef_)
print(reg2.intercept_)
md_probs = reg2.predict(test_onehot)
#print("Mean squared error: %.2f" % mean_squared_error(test_onehot, md_probs))

In [ ]:
additional_data = pd.read_csv("test.csv")

# Create a DataFrame with 'row ID' and predicted values
predicted_df = pd.DataFrame({
    'row ID': additional_data['row ID'],
    'price_doc': md_probs[:,0]  # Replace with the actual column name for your predictions
})

# Save the DataFrame to a new CSV file
predicted_df.to_csv('predictions12.csv', index=False)